# Tutorial 16: Deploy FastMCP Server to Azure Container Apps

## Production Deployment of MCP Servers

### What You'll Learn

In this tutorial, you'll deploy your FastMCP server to **Azure Container Apps** for production use. You'll learn:

- Dockerizing FastMCP servers
- Azure Container Apps setup and configuration
- Environment variable management
- Scaling and monitoring
- Secure deployment practices

**Use Case:** Deploy the Travel Booking MCP Server from Tutorial 15 to Azure Container Apps for production use.

**Duration:** 60 minutes

---

### Prerequisites

- Completed Tutorial 15 (FastMCP Server Basics)
- Azure subscription
- Docker Desktop installed
- Azure CLI installed
- Basic understanding of containers

## Part 1: Understanding Azure Container Apps

### What are Azure Container Apps?

**Azure Container Apps** is a fully managed serverless container service that enables you to run containerized applications without managing complex infrastructure.

**Key Features:**
- **Serverless**: Pay only for what you use
- **Auto-scaling**: Scale from 0 to many instances
- **Built-in HTTPS**: Automatic TLS certificates
- **Microservices**: Perfect for MCP servers
- **Integration**: Works with Azure services

### Architecture for MCP Server Deployment

```
┌─────────────────────────────────────────┐
│   Azure Container Registry (ACR)        │
│   └─ travel-mcp-server:latest          │
└────────────────┬────────────────────────┘
                 │ Pull Image
                 ↓
┌─────────────────────────────────────────┐
│   Azure Container App                    │
│   ┌─────────────────────────────────┐   │
│   │  FastMCP Server                 │   │
│   │  - Auto-scaling (0-10)          │   │
│   │  - HTTPS endpoint               │   │
│   │  - Health checks                │   │
│   └─────────────────────────────────┘   │
└────────────────┬────────────────────────┘
                 │
                 ↓
┌─────────────────────────────────────────┐
│   AI Agents (calling MCP tools)         │
└─────────────────────────────────────────┘
```

## Part 2: Dockerizing the FastMCP Server

First, we'll create a Dockerfile to containerize our MCP server.

In [ ]:
# Create Dockerfile for the MCP server
dockerfile_content = '''FROM python:3.11-slim

WORKDIR /app

# Install curl for health checks
RUN apt-get update && apt-get install -y curl && rm -rf /var/lib/apt/lists/*

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy server code
COPY travel_mcp_server.py .

# Expose port
EXPOSE 8000

# Health check - verify MCP endpoint is responding
HEALTHCHECK --interval=30s --timeout=10s --start-period=10s --retries=3 \\
  CMD curl -sf http://localhost:8000/mcp -X POST -H "Content-Type: application/json" -d '{"jsonrpc":"2.0","method":"ping","id":1}' || exit 1

# Run the server
CMD ["python", "-m", "uvicorn", "travel_mcp_server:app", "--host", "0.0.0.0", "--port", "8000"]
'''

# Create requirements.txt (using latest compatible versions)
requirements_content = '''fastmcp
pydantic
uvicorn[standard]
httpx
python-dotenv
'''

# Write files
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

print("Created: Dockerfile")
print("Created: requirements.txt")

Created: Dockerfile
Created: requirements.txt


### Build and Test Docker Image Locally

Before deploying to Azure, let's build and test the Docker image locally.

In [ ]:
# Build the Docker image for Azure (linux/amd64)
# Note: Azure Container Apps requires linux/amd64 architecture
import subprocess

print("Building Docker image for Azure (linux/amd64)...")
print("This may take a few minutes...\n")

result = subprocess.run(
    ["docker", "build", "--platform", "linux/amd64", "-t", "travel-mcp-server:latest", "."],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Docker image built successfully!")
    print("\nImage details:")
    print("  - Name: travel-mcp-server:latest")
    print("  - Platform: linux/amd64 (required for Azure)")
else:
    print("❌ Build failed:")
    print(result.stderr)

Building Docker image...
✓ Docker image built successfully

To run locally:
  docker run -p 8000:8000 travel-mcp-server:latest
✓ Docker image built successfully

To run locally:
  docker run -p 8000:8000 travel-mcp-server:latest


## Part 3: Azure Resource Setup

Now let's set up the Azure resources needed for deployment:
1. Resource Group
2. Azure Container Registry (ACR)
3. Container Apps Environment
4. Container App

In [4]:
# Configuration variables
import os
from dotenv import load_dotenv

load_dotenv()

# Azure configuration
RESOURCE_GROUP = "rg-mcp-servers"
LOCATION = "eastus"
ACR_NAME = "mcpserversacr"  # Must be globally unique, lowercase alphanumeric only
CONTAINER_APP_ENV = "mcp-servers-env"
CONTAINER_APP_NAME = "travel-mcp-server"

print("Azure Configuration:")
print(f"  Resource Group: {RESOURCE_GROUP}")
print(f"  Location: {LOCATION}")
print(f"  ACR Name: {ACR_NAME}")
print(f"  Container App Environment: {CONTAINER_APP_ENV}")
print(f"  Container App Name: {CONTAINER_APP_NAME}")

Azure Configuration:
  Resource Group: rg-mcp-servers
  Location: eastus
  ACR Name: mcpserversacr
  Container App Environment: mcp-servers-env
  Container App Name: travel-mcp-server


In [ ]:
# Create deployment script
deployment_script = f'''#!/bin/bash
set -e

echo "=== Azure MCP Server Deployment ==="

# 1. Create Resource Group
echo "Creating resource group..."
az group create \\
  --name {RESOURCE_GROUP} \\
  --location {LOCATION}

# 2. Create Azure Container Registry
echo "Creating Azure Container Registry..."
az acr create \\
  --resource-group {RESOURCE_GROUP} \\
  --name {ACR_NAME} \\
  --sku Basic \\
  --admin-enabled true

# 3. Login to ACR
echo "Logging in to ACR..."
az acr login --name {ACR_NAME}

# 4. Tag and push image to ACR
echo "Tagging and pushing image..."
docker tag travel-mcp-server:latest {ACR_NAME}.azurecr.io/travel-mcp-server:latest
docker push {ACR_NAME}.azurecr.io/travel-mcp-server:latest

# 5. Get ACR credentials
echo "Getting ACR credentials..."
ACR_USERNAME=$(az acr credential show --name {ACR_NAME} --query username -o tsv)
ACR_PASSWORD=$(az acr credential show --name {ACR_NAME} --query passwords[0].value -o tsv)

# 6. Create Container Apps Environment
echo "Creating Container Apps Environment..."
az containerapp env create \\
  --name {CONTAINER_APP_ENV} \\
  --resource-group {RESOURCE_GROUP} \\
  --location {LOCATION}

# 7. Deploy Container App
echo "Deploying Container App..."
az containerapp create \\
  --name {CONTAINER_APP_NAME} \\
  --resource-group {RESOURCE_GROUP} \\
  --environment {CONTAINER_APP_ENV} \\
  --image {ACR_NAME}.azurecr.io/travel-mcp-server:latest \\
  --target-port 8000 \\
  --ingress external \\
  --registry-server {ACR_NAME}.azurecr.io \\
  --registry-username $ACR_USERNAME \\
  --registry-password $ACR_PASSWORD \\
  --cpu 0.5 \\
  --memory 1.0Gi \\
  --min-replicas 0 \\
  --max-replicas 10

# 8. Get the application URL
echo "Getting application URL..."
APP_URL=$(az containerapp show \\
  --name {CONTAINER_APP_NAME} \\
  --resource-group {RESOURCE_GROUP} \\
  --query properties.configuration.ingress.fqdn -o tsv)

echo "=== Deployment Complete ==="
echo "MCP Server URL: https://$APP_URL"
echo "Test with: curl https://$APP_URL/health"
'''

# Save deployment script
with open('deploy_to_azure.sh', 'w') as f:
    f.write(deployment_script)

# Make it executable
import stat
os.chmod('deploy_to_azure.sh', os.stat('deploy_to_azure.sh').st_mode | stat.S_IEXEC)

print("Created: deploy_to_azure.sh")
print("\nTo deploy, run:")
print("  ./deploy_to_azure.sh")
print("\n⚠️  IMPORTANT: After deployment completes:")
print("  1. Copy the 'MCP Server URL' from the deployment output")
print("  2. Add this to your .env file:")
print("     MCP_SERVER_URL=https://your-actual-app-url.azurecontainerapps.io")
print("  3. Then run the testing cells below")

Created: deploy_to_azure.sh

To deploy, run:
  ./deploy_to_azure.sh


In [14]:
# Optional: Get the deployed Container App URL if already deployed
# Run this cell if you already deployed and need to retrieve the URL

import subprocess

try:
    result = subprocess.run(
        [
            "az", "containerapp", "show",
            "--name", CONTAINER_APP_NAME,
            "--resource-group", RESOURCE_GROUP,
            "--query", "properties.configuration.ingress.fqdn",
            "-o", "tsv"
        ],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0 and result.stdout.strip():
        app_url = result.stdout.strip()
        print(f"✅ Found deployed Container App!")
        print(f"\nMCP Server URL: https://{app_url}")
        print(f"\n📝 Add this to your .env file:")
        print(f"MCP_SERVER_URL=https://{app_url}")
        print(f"\n🧪 Test with: curl https://{app_url}/health")
    else:
        print("ℹ️  Container App not found or not yet deployed")
        print("Run the deployment script first: ./deploy_to_azure.sh")
except FileNotFoundError:
    print("ℹ️  Azure CLI not found. Please install it to retrieve the URL.")
    print("Or manually get the URL from Azure Portal")
except Exception as e:
    print(f"ℹ️  Could not retrieve URL: {e}")
    print("You can find the URL in Azure Portal under Container Apps")

ℹ️  Container App not found or not yet deployed
Run the deployment script first: ./deploy_to_azure.sh


## Part 4: Configuring Environment Variables and Secrets

For production deployments, we need to manage sensitive configuration securely.

In [6]:
# Script to update container app with environment variables
update_env_script = f'''#!/bin/bash

# Update container app with environment variables
echo "Updating environment variables..."

az containerapp update \\
  --name {CONTAINER_APP_NAME} \\
  --resource-group {RESOURCE_GROUP} \\
  --set-env-vars \\
    "API_VERSION=v1" \\
    "LOG_LEVEL=INFO" \\
    "MAX_CONCURRENT_REQUESTS=100" \\
  --secrets \\
    "api-key=your-secret-api-key-here"

echo "Environment variables updated"
'''

with open('update_env_vars.sh', 'w') as f:
    f.write(update_env_script)

os.chmod('update_env_vars.sh', os.stat('update_env_vars.sh').st_mode | stat.S_IEXEC)

print("Created: update_env_vars.sh")
print("\nBest Practices for Environment Variables:")
print("  - Use Azure Key Vault for sensitive data")
print("  - Never commit secrets to git")
print("  - Use managed identities when possible")
print("  - Rotate secrets regularly")

Created: update_env_vars.sh

Best Practices for Environment Variables:
  - Use Azure Key Vault for sensitive data
  - Never commit secrets to git
  - Use managed identities when possible
  - Rotate secrets regularly


## Part 5: Scaling Configuration

Azure Container Apps can auto-scale based on HTTP traffic, CPU, memory, or custom metrics.

In [7]:
# Configure scaling rules
scaling_script = f'''#!/bin/bash

# Configure HTTP-based autoscaling
echo "Configuring autoscaling..."

az containerapp update \\
  --name {CONTAINER_APP_NAME} \\
  --resource-group {RESOURCE_GROUP} \\
  --min-replicas 0 \\
  --max-replicas 10 \\
  --scale-rule-name http-rule \\
  --scale-rule-type http \\
  --scale-rule-http-concurrency 50

echo "Scaling configured:"
echo "  Min replicas: 0 (scale to zero when idle)"
echo "  Max replicas: 10"
echo "  HTTP concurrency: 50 requests per replica"
'''

with open('configure_scaling.sh', 'w') as f:
    f.write(scaling_script)

os.chmod('configure_scaling.sh', os.stat('configure_scaling.sh').st_mode | stat.S_IEXEC)

print("Created: configure_scaling.sh")
print("\nScaling Options:")
print("  - HTTP: Scale based on concurrent HTTP requests")
print("  - CPU: Scale based on CPU utilization")
print("  - Memory: Scale based on memory usage")
print("  - Custom: Scale based on Azure Monitor metrics")
print("  - Scale to Zero: Reduce costs when idle")

Created: configure_scaling.sh

Scaling Options:
  - HTTP: Scale based on concurrent HTTP requests
  - CPU: Scale based on CPU utilization
  - Memory: Scale based on memory usage
  - Custom: Scale based on Azure Monitor metrics
  - Scale to Zero: Reduce costs when idle


## Part 6: Monitoring and Testing

After deployment, let's test the MCP server and set up monitoring.

In [ ]:
# Test the deployed MCP server using MCP protocol
import httpx
import asyncio
import json

async def test_deployed_server(base_url):
    """Test the deployed MCP server using MCP JSON-RPC protocol"""
    print(f"Testing MCP server at: {base_url}\n")
    
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream"
    }
    
    async with httpx.AsyncClient() as client:
        # Test 1: Initialize MCP connection
        print("1. MCP Initialize...")
        try:
            init_request = {
                "jsonrpc": "2.0",
                "method": "initialize",
                "params": {
                    "protocolVersion": "2024-11-05",
                    "capabilities": {},
                    "clientInfo": {"name": "test-client", "version": "1.0"}
                },
                "id": 1
            }
            response = await client.post(
                f"{base_url}/mcp",
                json=init_request,
                headers=headers,
                timeout=15.0
            )
            if response.status_code == 200:
                # Parse SSE response
                lines = response.text.strip().split('\n')
                for line in lines:
                    if line.startswith('data: '):
                        data = json.loads(line[6:])
                        if 'result' in data:
                            server_info = data['result'].get('serverInfo', {})
                            print(f"   ✓ Connected to: {server_info.get('name', 'Unknown')}")
                            print(f"   ✓ Version: {server_info.get('version', 'Unknown')}")
            else:
                print(f"   ✗ Initialize failed: {response.status_code}")
        except Exception as e:
            print(f"   ✗ Error: {e}")
        
        # Test 2: List available tools
        print("\n2. List Available Tools...")
        try:
            list_tools_request = {
                "jsonrpc": "2.0",
                "method": "tools/list",
                "params": {},
                "id": 2
            }
            response = await client.post(
                f"{base_url}/mcp",
                json=list_tools_request,
                headers=headers,
                timeout=15.0
            )
            if response.status_code == 200:
                lines = response.text.strip().split('\n')
                for line in lines:
                    if line.startswith('data: '):
                        data = json.loads(line[6:])
                        if 'result' in data:
                            tools = data['result'].get('tools', [])
                            print(f"   ✓ Found {len(tools)} tools:")
                            for tool in tools:
                                print(f"      - {tool.get('name', 'unknown')}")
            else:
                print(f"   ✗ Failed to list tools: {response.status_code}")
        except Exception as e:
            print(f"   ✗ Error: {e}")
        
        # Test 3: Call a tool (search_flights)
        print("\n3. Test Tool Call (search_flights)...")
        try:
            call_tool_request = {
                "jsonrpc": "2.0",
                "method": "tools/call",
                "params": {
                    "name": "search_flights",
                    "arguments": {
                        "origin": "New York",
                        "destination": "London",
                        "departure_date": "2025-12-15",
                        "passengers": 2
                    }
                },
                "id": 3
            }
            response = await client.post(
                f"{base_url}/mcp",
                json=call_tool_request,
                headers=headers,
                timeout=30.0
            )
            if response.status_code == 200:
                lines = response.text.strip().split('\n')
                for line in lines:
                    if line.startswith('data: '):
                        data = json.loads(line[6:])
                        if 'result' in data:
                            content = data['result'].get('content', [])
                            if content:
                                text_content = content[0].get('text', '')
                                flights = json.loads(text_content) if text_content else []
                                print(f"   ✓ Tool executed successfully")
                                print(f"   ✓ Found {len(flights)} flights")
            else:
                print(f"   ✗ Tool call failed: {response.status_code}")
        except Exception as e:
            print(f"   ✗ Error: {e}")

# Note: Replace with your actual Container App URL after deployment
print("After deployment, test your server with:")
print('await test_deployed_server("https://your-app-url.azurecontainerapps.io")')

After deployment, test your server with:
await test_deployed_server("https://your-app-url.azurecontainerapps.io")


## Part 7: Using Deployed MCP Server with Azure AI Foundry Agent

Now that your MCP server is deployed, let's create an Azure AI Foundry agent that uses it as a hosted MCP tool.

In [9]:
# Import required libraries for Azure AI Foundry agent
from agent_framework import HostedMCPTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv
import asyncio

# Load environment variables
load_dotenv()

print("Azure AI Agent Framework imported successfully")
print("\nRequired environment variables:")
print("  - AZURE_AI_PROJECT_ENDPOINT")
print("  - AZURE_AI_MODEL_DEPLOYMENT_NAME")
print("  - MCP_SERVER_URL (your deployed Container App URL)")

Azure AI Agent Framework imported successfully

Required environment variables:
  - AZURE_AI_PROJECT_ENDPOINT
  - AZURE_AI_MODEL_DEPLOYMENT_NAME
  - MCP_SERVER_URL (your deployed Container App URL)


In [10]:
# Verify environment configuration
import os

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
mcp_server_url = os.getenv("MCP_SERVER_URL", "https://your-app.azurecontainerapps.io")

if not project_endpoint or not model_deployment:
    print("WARNING: Missing required Azure AI environment variables:")
    print("   Please set in your .env file:")
    print("   - AZURE_AI_PROJECT_ENDPOINT")
    print("   - AZURE_AI_MODEL_DEPLOYMENT_NAME")
    print("\n   See Tutorial 01b for Azure AI Foundry setup")
else:
    print("Azure AI configuration loaded")
    print(f"  Endpoint: {project_endpoint}")
    print(f"  Model: {model_deployment}")
    print(f"\nMCP Server URL: {mcp_server_url}")
    print("\nNote: Update MCP_SERVER_URL in .env with your actual Container App URL")

Azure AI configuration loaded
  Endpoint: https://gk-agent-framework-project.services.ai.azure.com/api/projects/agentframworkProject
  Model: gpt-4.1

MCP Server URL: https://your-app.azurecontainerapps.io

Note: Update MCP_SERVER_URL in .env with your actual Container App URL


In [11]:
# Create Azure AI Foundry agent with hosted MCP tool
async def create_travel_agent_with_mcp():
    """Create an Azure AI Foundry agent that uses the deployed MCP server"""
    
    print("=" * 70)
    print("Creating Azure AI Foundry Agent with Hosted MCP Server")
    print("=" * 70)
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        # Enable Azure AI observability (optional but recommended)
        await chat_client.setup_azure_ai_observability()
        print("\nAzure AI observability enabled")
        
        # Create agent with hosted MCP tool
        print(f"\nConnecting to MCP server: {mcp_server_url}")
        agent = chat_client.create_agent(
            name="TravelBookingAgent",
            instructions="""
            You are a helpful travel booking assistant with access to real-time travel tools.
            
            You can:
            - Search for flights between cities
            - Check hotel availability and pricing
            - Convert currencies for budget planning
            
            When helping users:
            1. Use the MCP tools to get accurate, real-time information
            2. Provide clear, organized recommendations
            3. Help with currency conversions when discussing costs
            4. Be friendly, professional, and detail-oriented
            """,
            tools=HostedMCPTool(
                name="Travel Booking MCP Server",
                url=mcp_server_url,
                approval_mode="never_require",  # Auto-approve for demo
                # For production, consider "always_require" for manual approval
            ),
        )
        
        print(f"Agent created: {agent.name}")
        print("MCP tools connected and ready")
        
        return agent, chat_client

# Note: We'll use this function in the next cell
print("Agent creation function defined")
print("Run the next cell to test the agent")

Agent creation function defined
Run the next cell to test the agent


In [12]:
# Test the agent with a simple travel query
async def test_foundry_agent_simple():
    """Test the Azure AI Foundry agent with a simple query"""
    
    print("\n" + "=" * 70)
    print("TEST 1: Simple Flight Search via Foundry Agent")
    print("=" * 70)
    
    user_query = "Find me flights from Seattle to Tokyo on December 1st for 1 passenger."
    print(f"\nUser: {user_query}\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        await chat_client.setup_azure_ai_observability()
        
        # Create agent with hosted MCP tool
        agent = chat_client.create_agent(
            name="TravelBookingAgent",
            instructions="You are a travel assistant. Use the MCP tools to search for flights and hotels.",
            tools=HostedMCPTool(
                name="Travel Booking MCP Server",
                url=mcp_server_url,
                approval_mode="never_require",
            ),
        )
        
        # Run the query
        result = await agent.run(user_query)
        
        print(f"Agent: {result.text}\n")
    
    print("=" * 70)

# Run the test if environment is configured
if project_endpoint and model_deployment:
    if mcp_server_url and "your-app" not in mcp_server_url:
        await test_foundry_agent_simple()
    else:
        print("\nPlease update MCP_SERVER_URL in your .env file with your actual Container App URL")
        print("Format: https://your-app-name.azurecontainerapps.io")
else:
    print("WARNING: Please configure Azure AI environment variables first")


Please update MCP_SERVER_URL in your .env file with your actual Container App URL
Format: https://your-app-name.azurecontainerapps.io


In [13]:
# Test the agent with a complex multi-tool query
async def test_foundry_agent_complex():
    """Test the Azure AI Foundry agent with a complex multi-tool query"""
    
    print("\n" + "=" * 70)
    print("TEST 2: Complex Multi-Tool Query via Foundry Agent")
    print("=" * 70)
    
    user_query = """
    I'm planning a trip to Paris from New York. Can you help me with:
    1. Flights for January 15th for 2 passengers
    2. Hotels for 4 nights (Jan 15-19)
    3. Total estimated cost in Euros (my budget is $3000 USD)
    """
    
    print(f"\nUser: {user_query}\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        await chat_client.setup_azure_ai_observability()
        
        # Create agent with hosted MCP tool
        agent = chat_client.create_agent(
            name="TravelBookingAgent",
            instructions="""
            You are a comprehensive travel assistant. Use the MCP tools to:
            - Search for flights
            - Check hotel availability
            - Convert currencies
            
            Provide detailed recommendations with pricing and availability.
            """,
            tools=HostedMCPTool(
                name="Travel Booking MCP Server",
                url=mcp_server_url,
                approval_mode="never_require",
            ),
        )
        
        # Run the complex query - agent will call multiple MCP tools
        result = await agent.run(user_query)
        
        print(f"Agent: {result.text}\n")
    
    print("=" * 70)

# Run the test if environment is configured
if project_endpoint and model_deployment:
    if mcp_server_url and "your-app" not in mcp_server_url:
        await test_foundry_agent_complex()
    else:
        print("\nPlease update MCP_SERVER_URL in your .env file with your actual Container App URL")
else:
    print("WARNING: Please configure Azure AI environment variables first")


Please update MCP_SERVER_URL in your .env file with your actual Container App URL


## Key Differences: Local vs Hosted MCP Integration

### Local MCP Integration (Tutorial 15)
```python
# Direct function calls - tools run in the same process
from travel_mcp_server import _search_flights, _check_hotel_availability

agent = client.create_agent(
    tools=[_search_flights, _check_hotel_availability]
)
```

**Pros:**
- Simple for development and testing
- No network latency
- Easy to debug

**Cons:**
- Tools run in agent process (scaling issues)
- Can't share tools across multiple agents
- Limited security isolation

### Hosted MCP Integration (Tutorial 16)
```python
# Remote MCP server - tools run on Azure Container Apps
agent = client.create_agent(
    tools=HostedMCPTool(
        name="Travel Booking MCP Server",
        url="https://your-app.azurecontainerapps.io",
        approval_mode="never_require"
    )
)
```

**Pros:**
- Tools run independently (better scaling)
- Share tools across multiple agents
- Strong security isolation
- Azure manages infrastructure
- Built-in monitoring and logging

**Cons:**
- Network latency for tool calls
- Requires deployment and maintenance
- Additional Azure costs

### When to Use Each Approach

**Use Local MCP (Tutorial 15)** for:
- Development and testing
- Simple, single-agent applications
- Quick prototyping

**Use Hosted MCP (Tutorial 16)** for:
- Production deployments
- Multi-agent systems
- Enterprise applications
- Tools that need isolation or scaling

## Summary and Key Takeaways

### What You've Learned

1. **Dockerization**: How to containerize FastMCP servers with proper configuration

2. **Azure Container Apps**: Deploying containerized MCP servers to Azure with:
   - Auto-scaling (including scale-to-zero)
   - Built-in HTTPS and custom domains
   - Health checks and monitoring
   
3. **Production Best Practices**:
   - Environment variable management
   - Secrets handling with Azure Key Vault
   - Scaling strategies
   - Monitoring and logging

4. **Azure AI Foundry Integration**: Connecting deployed MCP servers to AI agents using:
   - HostedMCPTool for remote MCP server connections
   - Approval workflows for tool execution control
   - Azure AI observability for monitoring

5. **CI/CD**: Foundation for automated deployment pipelines

### Cost Optimization

- **Scale to Zero**: Pay nothing when idle
- **Right-sizing**: Start with minimal resources (0.5 CPU, 1GB RAM)
- **Auto-scaling**: Only pay for what you use

### Next Steps

In **Tutorial 17** (Logic App as MCP Server), you'll learn how to:
- Create serverless MCP servers using Azure Logic Apps
- Integrate with Azure services without code
- Build event-driven MCP workflows
- Connect multiple Azure services as MCP tools

### Additional Resources

- **Azure Container Apps Docs**: https://learn.microsoft.com/azure/container-apps/
- **Container Apps Pricing**: https://azure.microsoft.com/pricing/details/container-apps/
- **Scaling in Container Apps**: https://learn.microsoft.com/azure/container-apps/scale-app

---

**Congratulations!** Your MCP server is now running in production on Azure with enterprise-grade scaling, monitoring, and security.